In [1]:
import os
from PIL import Image
import pandas as pd

In [2]:
DO_SAMPLE = False
# DO_SAMPLE = True
NUM_SAMPLE = 1000
dir_path = (
    "/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search/"
)
image_path = f"{dir_path}/crop_image"
image_list = os.listdir(image_path)
image_list = [
    os.path.join(image_path, image) for image in image_list if not image.startswith(".")
]
# sort
image_list.sort()
print(len(image_list))
image_list[:5], image_list[-5:]

11118


(['/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/10.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/100.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/1000.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/10000.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/10001.png'],
 ['/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/9990.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/9991.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/9992.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//crop_image/9993.png',
  '/Users/user/Documents/Coding/cro_location_intelligence/reverse_image_search//cr

In [3]:
df_path = f"{dir_path}/7-11 Location for Ford.xlsx"
df = pd.read_excel(df_path)
print(df.shape)
df.head()

(11118, 4)


,store_id,prov_namt,latitude,longitude
0,1025,กรุงเทพมหานคร,13.838724,100.575318
1,1026,ปทุมธานี,14.036545,100.733520
2,1028,ตรัง,7.567873,99.614094
3,1032,กรุงเทพมหานคร,13.719625,100.442061
4,1033,กรุงเทพมหานคร,13.768390,100.721288


In [4]:
# find duplicate store_id in df
duplicate_store_id = df[df.duplicated(subset=["store_id"], keep=False)]
duplicate_store_id = duplicate_store_id.store_id.tolist()
duplicate_store_id = list(set(duplicate_store_id))
print(duplicate_store_id)
print(len(duplicate_store_id))

[]
0


In [5]:
import numpy as np

# add new column name embedding with nan
df["embedding"] = [[]] * len(df)
df.head()

,store_id,prov_namt,latitude,longitude,embedding
0,1025,กรุงเทพมหานคร,13.838724,100.575318,[]
1,1026,ปทุมธานี,14.036545,100.733520,[]
2,1028,ตรัง,7.567873,99.614094,[]
3,1032,กรุงเทพมหานคร,13.719625,100.442061,[]
4,1033,กรุงเทพมหานคร,13.768390,100.721288,[]


In [6]:
# check is store_id is unique
df["store_id"].is_unique

True

In [7]:
# do model inference

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )

else:
    device = torch.device("mps")
device

device(type='mps')

In [9]:
import timm
from geo_utils import (
    crop_middle_image_from_side_size,
    apply_circle_mask,
)

model = timm.create_model(
    "maxvit_tiny_tf_512.in1k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
).to(
    device
)  # Move the model to the GPU
model = model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)


def preprocess_image(image: Image.Image) -> Image.Image:
    # image = crop_middle_image_from_side_size(image, 2400, 1600)
    image = apply_circle_mask(image)
    image = image.resize((512, 512))
    return image


preprocessed_image_folder = f"{dir_path}/crop_image"


def predict(model, transforms, image_path, preprocessed_image_path, device=device):
    image = Image.open(image_path).convert("RGB")
    preprocessed_image = preprocess_image(image)
    image_tensor = (
        transforms(preprocessed_image).unsqueeze(0).to(device)
    )  # Move input data to the GPU
    output = model(image_tensor)

    # or equivalently (without needing to set num_classes=0)
    output = model.forward_features(image_tensor)
    output = model.forward_head(output, pre_logits=True)
    embedding = output.detach().cpu().numpy()[0]
    preprocessed_image.save(preprocessed_image_path)
    # save preprocessed_image
    return embedding  # Move the result back to the CPU for further processing if needed

In [10]:
# image_path = f"{dir_path}/data/full_image/1026.png"
image_path = f"{dir_path}/crop_image/1025.png"
preprocessed_image_path = os.path.join(
    preprocessed_image_folder, os.path.basename(image_path)
)
result = predict(
    model,
    transforms,
    image_path,
    preprocessed_image_path,
)
result

array([-0.5909128 , -0.15493128,  0.10268707, -0.20271835, -0.45178953,
        0.5089837 , -0.21708447,  0.18938565, -0.01838017,  0.38833663,
       -0.60113746, -0.7828741 , -0.8766293 , -0.38296315,  0.17193179,
        0.6206502 , -0.2866303 ,  0.21968898,  0.73409843, -0.85176265,
        0.6758186 , -0.57760835, -0.23458506,  0.40565485,  0.2208155 ,
       -0.12820598, -0.29737085, -0.4085765 , -0.0824457 , -0.8832159 ,
       -0.23987757,  0.02702179, -0.2515429 ,  0.67758733,  0.62235147,
       -0.11359639,  0.16227286,  0.16672984, -0.21988417,  0.21630779,
        0.6808744 ,  0.44098097, -0.65373325, -0.24939227, -0.47979563,
        0.49296466, -0.32926127,  0.13914368, -0.17216931, -0.29970512,
       -0.1548718 ,  0.11567356, -0.37361157, -0.30484638, -0.45426822,
        0.21772666, -0.03108997, -0.43859777, -0.10836279, -0.37685284,
        0.13366553,  0.57591873, -0.7724935 , -0.40570304, -0.48875692,
        0.10378516,  0.8892232 , -0.25500432,  0.5513068 , -0.62

In [11]:
index = 1
df.loc[index, "embedding"], pd.isna(df.loc[index, "embedding"])

([], array([], dtype=bool))

In [12]:
save_path = "./df_embedding.pkl"
import os
import pandas as pd

if os.path.exists(save_path):
    print("Load success df_embedding.pkl")
    df = pd.read_pickle(save_path)

In [13]:
df.head()

,store_id,prov_namt,latitude,longitude,embedding
0,1025,กรุงเทพมหานคร,13.838724,100.575318,[]
1,1026,ปทุมธานี,14.036545,100.733520,[]
2,1028,ตรัง,7.567873,99.614094,[]
3,1032,กรุงเทพมหานคร,13.719625,100.442061,[]
4,1033,กรุงเทพมหานคร,13.768390,100.721288,[]


In [14]:
# count embedding that is []
df["embedding"].apply(lambda x: x == []).sum()

11118

In [15]:
if DO_SAMPLE is True:
    # select first 1000
    df = df[:NUM_SAMPLE]
    print(df.shape)

In [16]:
store_id_list = df["store_id"]
image_folder = preprocessed_image_folder
from tqdm import tqdm

image_paths = []
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    store_id = df.loc[index, "store_id"]
    # print(store_id)

    image_path = os.path.join(image_folder, f"{store_id}.png")
    if not os.path.exists(image_path):
        continue
    # is nan or not
    # print('df.loc[index, "embedding"]', df.loc[index, "embedding"])
    if df.loc[index, "embedding"] != []:
        continue
    preprocessed_image_path = os.path.join(
        preprocessed_image_folder, os.path.basename(image_path)
    )
    # if os.path.exists(preprocessed_image_path):
    #     continue
    image_paths.append(preprocessed_image_path)
batch_size = 8  # Choose an appropriate batch size

for i in tqdm(
    range(0, len(image_paths), batch_size), desc="Processing Images", leave=False
):
    batch_paths = image_paths[i : i + batch_size]

    batch_images = [Image.open(path).convert("RGB") for path in batch_paths]
    preprocessed_images = [preprocess_image(image) for image in batch_images]
    # image_tensors = torch.stack(
    #     [transforms(image).unsqueeze(0) for image in preprocessed_images]
    # ).to(device)
    image_tensors = (
        torch.stack([transforms(image).unsqueeze(0) for image in preprocessed_images])
        .squeeze(1)
        .to(device)
    )

    # Move input data to the GPU
    output = model(image_tensors)

    # or equivalently (without needing to set num_classes=0)
    output = model.forward_features(image_tensors)
    output = model.forward_head(output, pre_logits=True)
    embeddings = output.detach().cpu().numpy()

    for j, embedding in enumerate(embeddings):
        index = i + j  # Assuming 'index' is the index to update in the DataFrame
        df.embedding[index] = embedding.tolist()

Processing rows:   0%|          | 0/11118 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/1390 [00:00<?, ?it/s]/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_88609/1286111549.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embedding[index] = embedding.tolist()


In [17]:
# save df to pickle
df.to_pickle(save_path)

In [18]:
# count embedding that is []
df["embedding"].apply(lambda x: x == []).sum()

0